<a href="https://colab.research.google.com/github/eldiablo-data/tensorflow2learning/blob/master/notebooks/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Tensorflow Demo for MNIST Dataset

In [53]:
import tensorflow as tf
print(tf.__version__)

2.5.0


## Loading MNIST data
MNIST dataset is already available within tensorflow package. The numbers are represented as 28x28 pixels stored as array. The pixels have value ranging from 0 to 255 which represents the gray scale from white to black respectively. The load() function returns a tuple of training and test sets.

In [54]:
mnist = tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test)= mnist.load_data() 

In [55]:
x_train_scaled = x_train/255
x_test_scaled = x_test/255

## Model initialization

In [56]:
model = tf.keras.models.Sequential()

The above model is a Sequential model. Here, each layer gets input from the previous layer and passed output to next layer. As alternative to Sequential model, the Functional API of Keras allows users to define more complex graph or layers where a layer can get input from more than one layer and pass outputs to multiple layers.

In [57]:
model.add(tf.keras.layers.Flatten(input_shape=(28,28)))

Usually images are flattened into a vector to represent each input image as a row of the dataset.

In [58]:
model.add(tf.keras.layers.Dense(units=128,activation='relu'))

Dense layer is a regular densely-connected NN layer. The number of units is a hyper-parameter selected by experience. 
The output of a dense layer is calculated by `output = activation(dot(input, kernel) + bias)`. Here the ReLU or Rectified Linear Unit activation function is used. ReLU is half rectified function and it returns 0 for all negative inputs and for positive inputs it increases monotonically. 

In [59]:
model.add(tf.keras.layers.Dropout(0.2))

The dropout layer sets input units to 0 with the given rate. This is used to prevent overfitting. The shape can also be modified into a 1D tensor mask. And seed values can also be fixed. The dropout is active only when `training=True`. During inference, it is not used. During `model.fit()` training is True by default.

In [60]:
model.add(tf.keras.layers.Dense(10,'softmax'))

Softmax converts a vector of values into a probablity distribution. Usually it is used in output layers as it can be used to interpret probablity distributions. Here 10 is used as we have 0-9 as the labels. 

## Compile the Model

During this step, the configurations of the model are assigned. Optimizers are described in detail on later sections. In this example, `adam`optimizer is used.

In [61]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

`sparse_categorial_crossentropy` function used here is a loss function that is useful when the output prediction is a sparse array. In this case, the one-hot encoding 
produces 1 for the right integer and 0 for all other indices. The `sparse_categorial_crossentropy` function only computes the loss for the k<sup>th</sup> index and ignores the rest.
The cross entrophy loss for the rest of the positions would anyways be 0 and just summing up 0s is redundant. 


In [62]:
trainingProgress = model.fit(x=x_train_scaled,y=y_train, validation_data=(x_test_scaled,y_test),epochs=10)

Epoch 1/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2985 - accuracy: 0.9121 - val_loss: 0.1425 - val_accuracy: 0.9573
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1471 - accuracy: 0.9568 - val_loss: 0.1000 - val_accuracy: 0.9712
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1086 - accuracy: 0.9666 - val_loss: 0.0857 - val_accuracy: 0.9748
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0918 - accuracy: 0.9715 - val_loss: 0.0765 - val_accuracy: 0.9766
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0772 - accuracy: 0.9758 - val_loss: 0.0775 - val_accuracy: 0.9751
Epoch 6/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0686 - accuracy: 0.9780 - val_loss: 0.0777 - val_accuracy: 0.9754
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0589 - accuracy: 0.9808 - val_loss: 0.0745 - val_accuracy:

## Model Evaluation


In [63]:
import plotly.graph_objects as go
import plotly.express as px



### Loss

In [64]:
lossPlot = go.Figure()
lossPlot.add_trace(go.Scatter(y=trainingProgress.history['loss'],name='loss'))
lossPlot.add_trace(go.Scatter(y=trainingProgress.history['val_loss'],name='val_loss'))

In [65]:
accuracyPlot = go.Figure()
accuracyPlot.add_trace(go.Scatter(y=trainingProgress.history['accuracy'],name='Training accuracy'))
accuracyPlot.add_trace(go.Scatter(y=trainingProgress.history['val_accuracy'],name='Testing accuracy'))

In [66]:
pred_y= model.predict(x_test).argmax(axis=1)
from pandas import crosstab as tab
confusion = tab(pred_y,y_test,rownames=['pred_y'],colnames=['y_test'])
confusion

y_test,0,1,2,3,4,5,6,7,8,9
pred_y,,,,,,,,,,
0,972,0,6,0,0,2,5,3,5,2
1,1,1128,1,0,0,0,3,4,2,4
2,2,5,1009,3,3,0,0,11,5,0
3,1,0,5,994,1,19,1,2,9,7
4,0,0,0,0,948,1,2,0,4,2
5,0,1,0,1,0,855,3,0,1,3
6,2,1,3,0,6,8,941,0,3,0
7,1,0,4,5,7,1,1,1004,6,5
8,1,0,4,2,2,3,1,0,936,2


In [67]:
px.imshow(confusion,zmax=20)